In [2]:
import pandas as pd
from model import DelayModel

# Cargar datos desde data.csv
data = pd.read_csv('../data/data.csv')

# Crear una instancia del modelo
model = DelayModel()

# Preprocesar los datos y entrenar el modelo
features, target = model.preprocess(data, target_column='delay')

# Guardar el modelo entrenado
model.fit(features, target)

print("Modelo entrenado y guardado exitosamente.")

C:\Users\jlorregoh\AppData\Local\Temp\ipykernel_15880\2546564056.py:5: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/data.csv')


Modelo entrenado y guardado exitosamente.


C:\Users\jlorregoh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
